In [1]:
## This file implements neural networks and random forest on p11presabsSTCC_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 90.85% for over-sampling data, 93.29% after improvement.
## And it is 94.62% for combination data and 93.55% with dropout and regularization.
## For random forest, the accuracy is 91.46% for over-sampling data and 92.47% for combination data.
## For random forest with cross-validation, the mean accuracy is 85.93% for over-sampling, and 95.78% for combination data.

In [2]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [3]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p11presabsSTCC_qual.csv')
df.shape

(255, 976)

In [4]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [5]:
df['pheno']

0      2
1      1
2      2
3      2
4      2
5      2
6      2
7      2
8      2
9      2
10     1
11     2
12     2
13     1
14     2
15     2
16     0
17     0
18     2
19     0
20     0
21     0
22     1
23     2
24     2
25     0
26     2
27     0
28     2
29     2
      ..
225    2
226    1
227    2
228    2
229    2
230    1
231    2
232    2
233    2
234    2
235    2
236    2
237    2
238    2
239    2
240    2
241    2
242    0
243    2
244    2
245    2
246    2
247    2
248    2
249    2
250    2
251    1
252    2
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [6]:
df['pheno'].value_counts()

2    182
1     48
0     25
Name: pheno, dtype: int64

In [7]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,ST,CC,pheno
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [8]:
df_clean = df.drop(columns=['id'])

In [9]:
df_clean.shape

(255, 975)

In [10]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,ST,CC,pheno
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,8,8,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,5,5,2


In [11]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 974) (255,)


In [12]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 182), (1, 182), (2, 182)]


Using TensorFlow backend.


In [13]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 131), (1, 111), (2, 67)]


In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
############# Fully-Connected Neural Network ################

In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/100
382/382 [==============================] - 0s 333us/step - loss: 3.5618 - accuracy: 0.3639 - val_loss: 1.4538 - val_accuracy: 0.4939
Epoch 2/100
382/382 [==============================] - 0s 75us/step - loss: 1.5206 - accuracy: 0.5157 - val_loss: 0.7870 - val_accuracy: 0.6585
Epoch 3/100
382/382 [==============================] - 0s 74us/step - loss: 1.1980 - accuracy: 0.6099 - val_loss: 0.8638 - val_accuracy: 0.6951
Epoch 4/100
382/382 [==============================] - 0s 84us/step - loss: 1.0690 - accuracy: 0.6387 - val_loss: 0.7135 - val_accuracy: 0.6890
Epoch 5/100
382/382 [==============================] - 0s 76us/step - loss: 0.7795 - accuracy: 0.6728 - val_loss: 0.6775 - val_accuracy: 0.6768
Epoch 6/100
382/382 [==============================] - 0s 80us/step - loss: 0.6808 - accuracy: 0.7016 - val_loss: 0.6243 - val_accuracy: 0.7134
Epoch 7/100
382/382 [==============================] - 0s 80us/step - loss: 0.6387 - accu

382/382 [==============================] - 0s 71us/step - loss: 0.1818 - accuracy: 0.9450 - val_loss: 0.3495 - val_accuracy: 0.9085
Epoch 58/100
382/382 [==============================] - ETA: 0s - loss: 0.2306 - accuracy: 0.90 - 0s 69us/step - loss: 0.1914 - accuracy: 0.9372 - val_loss: 0.3887 - val_accuracy: 0.8598
Epoch 59/100
382/382 [==============================] - 0s 74us/step - loss: 0.2074 - accuracy: 0.9215 - val_loss: 0.4181 - val_accuracy: 0.8598
Epoch 60/100
382/382 [==============================] - 0s 81us/step - loss: 0.2920 - accuracy: 0.9241 - val_loss: 0.3753 - val_accuracy: 0.8476
Epoch 61/100
382/382 [==============================] - 0s 76us/step - loss: 0.2422 - accuracy: 0.9319 - val_loss: 0.5295 - val_accuracy: 0.8720
Epoch 62/100
382/382 [==============================] - 0s 74us/step - loss: 0.4048 - accuracy: 0.9005 - val_loss: 0.3853 - val_accuracy: 0.8659
Epoch 63/100
382/382 [==============================] - 0s 74us/step - loss: 0.3250 - accuracy: 0.913

In [21]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

164/164 [==============================] - 0s 75us/step
over-sampling test accuracy: 90.85%


In [22]:
#### improve neural network on over-sampling data with dropout
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [23]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 382 samples, validate on 164 samples
Epoch 1/1000
382/382 [==============================] - 0s 798us/step - loss: 4.1386 - accuracy: 0.4424 - val_loss: 1.2428 - val_accuracy: 0.5976
Epoch 2/1000
382/382 [==============================] - 0s 168us/step - loss: 3.8572 - accuracy: 0.5366 - val_loss: 1.0613 - val_accuracy: 0.6341
Epoch 3/1000
382/382 [==============================] - 0s 183us/step - loss: 3.3231 - accuracy: 0.5288 - val_loss: 1.0258 - val_accuracy: 0.6280
Epoch 4/1000
382/382 [==============================] - 0s 273us/step - loss: 2.7704 - accuracy: 0.5969 - val_loss: 0.9489 - val_accuracy: 0.6402
Epoch 5/1000
382/382 [==============================] - 0s 135us/step - loss: 3.2442 - accuracy: 0.5759 - val_loss: 0.9462 - val_accuracy: 0.6402
Epoch 6/1000
382/382 [==============================] - 0s 107us/step - loss: 3.3542 - accuracy: 0.5942 - val_loss: 0.8016 - val_accuracy: 0.6951
Epoch 7/1000
382/382 [==============================] - 0s 89us/step - loss: 3

Epoch 57/1000
382/382 [==============================] - 0s 80us/step - loss: 1.7341 - accuracy: 0.6937 - val_loss: 0.8157 - val_accuracy: 0.8354
Epoch 58/1000
382/382 [==============================] - 0s 77us/step - loss: 1.4399 - accuracy: 0.7539 - val_loss: 0.6992 - val_accuracy: 0.8537
Epoch 59/1000
382/382 [==============================] - 0s 93us/step - loss: 1.6479 - accuracy: 0.7120 - val_loss: 0.6712 - val_accuracy: 0.8537
Epoch 60/1000
382/382 [==============================] - 0s 115us/step - loss: 1.7576 - accuracy: 0.6911 - val_loss: 1.1029 - val_accuracy: 0.7866
Epoch 61/1000
382/382 [==============================] - 0s 104us/step - loss: 1.6683 - accuracy: 0.7330 - val_loss: 0.8495 - val_accuracy: 0.8476
Epoch 62/1000
382/382 [==============================] - 0s 89us/step - loss: 1.9228 - accuracy: 0.6414 - val_loss: 1.2519 - val_accuracy: 0.7866
Epoch 63/1000
382/382 [==============================] - 0s 84us/step - loss: 1.9786 - accuracy: 0.7016 - val_loss: 1.2201

Epoch 113/1000
382/382 [==============================] - 0s 90us/step - loss: 1.1922 - accuracy: 0.7382 - val_loss: 0.6914 - val_accuracy: 0.8659
Epoch 114/1000
382/382 [==============================] - 0s 103us/step - loss: 1.1074 - accuracy: 0.7513 - val_loss: 1.2951 - val_accuracy: 0.8598
Epoch 115/1000
382/382 [==============================] - 0s 98us/step - loss: 1.4651 - accuracy: 0.7016 - val_loss: 2.2496 - val_accuracy: 0.7805
Epoch 116/1000
382/382 [==============================] - 0s 111us/step - loss: 1.3902 - accuracy: 0.7592 - val_loss: 1.2273 - val_accuracy: 0.8232
Epoch 117/1000
382/382 [==============================] - 0s 97us/step - loss: 1.2165 - accuracy: 0.7330 - val_loss: 0.7044 - val_accuracy: 0.8963
Epoch 118/1000
382/382 [==============================] - 0s 127us/step - loss: 1.0821 - accuracy: 0.7277 - val_loss: 0.9844 - val_accuracy: 0.8537
Epoch 119/1000
382/382 [==============================] - 0s 105us/step - loss: 1.1941 - accuracy: 0.7723 - val_los

382/382 [==============================] - 0s 101us/step - loss: 1.3657 - accuracy: 0.7539 - val_loss: 1.6146 - val_accuracy: 0.8171
Epoch 169/1000
382/382 [==============================] - 0s 99us/step - loss: 1.3362 - accuracy: 0.7565 - val_loss: 0.8873 - val_accuracy: 0.8841
Epoch 170/1000
382/382 [==============================] - 0s 85us/step - loss: 0.9960 - accuracy: 0.7147 - val_loss: 0.6552 - val_accuracy: 0.9146
Epoch 171/1000
382/382 [==============================] - 0s 90us/step - loss: 1.1182 - accuracy: 0.7330 - val_loss: 0.8971 - val_accuracy: 0.8720
Epoch 172/1000
382/382 [==============================] - 0s 87us/step - loss: 0.9734 - accuracy: 0.7513 - val_loss: 0.8586 - val_accuracy: 0.8841
Epoch 173/1000
382/382 [==============================] - 0s 90us/step - loss: 0.7126 - accuracy: 0.7487 - val_loss: 0.5834 - val_accuracy: 0.8963
Epoch 174/1000
382/382 [==============================] - 0s 87us/step - loss: 0.8032 - accuracy: 0.7723 - val_loss: 0.6686 - val_ac

Epoch 224/1000
382/382 [==============================] - 0s 96us/step - loss: 0.6326 - accuracy: 0.7775 - val_loss: 0.8476 - val_accuracy: 0.9085
Epoch 225/1000
382/382 [==============================] - 0s 93us/step - loss: 0.5687 - accuracy: 0.8168 - val_loss: 0.6550 - val_accuracy: 0.9390
Epoch 226/1000
382/382 [==============================] - 0s 91us/step - loss: 0.8651 - accuracy: 0.7723 - val_loss: 0.8509 - val_accuracy: 0.8841
Epoch 227/1000
382/382 [==============================] - 0s 90us/step - loss: 0.5986 - accuracy: 0.8089 - val_loss: 0.5944 - val_accuracy: 0.9268
Epoch 228/1000
382/382 [==============================] - 0s 90us/step - loss: 0.7162 - accuracy: 0.7958 - val_loss: 1.0000 - val_accuracy: 0.9146
Epoch 229/1000
382/382 [==============================] - 0s 89us/step - loss: 0.7419 - accuracy: 0.7749 - val_loss: 0.7442 - val_accuracy: 0.9268
Epoch 230/1000
382/382 [==============================] - 0s 84us/step - loss: 0.6828 - accuracy: 0.7696 - val_loss: 0

...

382/382 [==============================] - 0s 124us/step - loss: 0.7191 - accuracy: 0.7487 - val_loss: 0.6046 - val_accuracy: 0.9329
Epoch 775/1000
382/382 [==============================] - 0s 156us/step - loss: 0.6076 - accuracy: 0.7775 - val_loss: 0.9367 - val_accuracy: 0.9390
Epoch 776/1000
382/382 [==============================] - 0s 204us/step - loss: 0.6625 - accuracy: 0.7906 - val_loss: 1.0777 - val_accuracy: 0.9207
Epoch 777/1000
382/382 [==============================] - 0s 111us/step - loss: 0.6675 - accuracy: 0.7775 - val_loss: 1.0731 - val_accuracy: 0.9329
Epoch 778/1000
382/382 [==============================] - 0s 96us/step - loss: 0.6662 - accuracy: 0.7801 - val_loss: 1.0551 - val_accuracy: 0.9207
Epoch 779/1000
382/382 [==============================] - 0s 87us/step - loss: 0.7940 - accuracy: 0.7723 - val_loss: 1.0081 - val_accuracy: 0.9329
Epoch 780/1000
382/382 [==============================] - 0s 84us/step - loss: 0.5128 - accuracy: 0.8220 - val_loss: 0.9242 - val

382/382 [==============================] - 0s 171us/step - loss: 0.4373 - accuracy: 0.8325 - val_loss: 0.9374 - val_accuracy: 0.9329
Epoch 830/1000
382/382 [==============================] - 0s 143us/step - loss: 0.5867 - accuracy: 0.8010 - val_loss: 0.8829 - val_accuracy: 0.9268
Epoch 831/1000
382/382 [==============================] - 0s 134us/step - loss: 0.5629 - accuracy: 0.8168 - val_loss: 0.8989 - val_accuracy: 0.9390
Epoch 832/1000
382/382 [==============================] - 0s 108us/step - loss: 0.5104 - accuracy: 0.7958 - val_loss: 0.8991 - val_accuracy: 0.9329
Epoch 833/1000
382/382 [==============================] - 0s 102us/step - loss: 0.6285 - accuracy: 0.7827 - val_loss: 0.9882 - val_accuracy: 0.9268
Epoch 834/1000
382/382 [==============================] - 0s 105us/step - loss: 0.4299 - accuracy: 0.8220 - val_loss: 0.9248 - val_accuracy: 0.9390
Epoch 835/1000
382/382 [==============================] - 0s 100us/step - loss: 0.5479 - accuracy: 0.7618 - val_loss: 0.9103 - 

382/382 [==============================] - 0s 83us/step - loss: 0.4741 - accuracy: 0.8194 - val_loss: 0.7525 - val_accuracy: 0.9329
Epoch 885/1000
382/382 [==============================] - 0s 100us/step - loss: 0.5007 - accuracy: 0.7801 - val_loss: 0.6306 - val_accuracy: 0.9329
Epoch 886/1000
382/382 [==============================] - 0s 97us/step - loss: 0.6942 - accuracy: 0.7853 - val_loss: 0.8066 - val_accuracy: 0.9329
Epoch 887/1000
382/382 [==============================] - 0s 100us/step - loss: 0.6899 - accuracy: 0.7670 - val_loss: 0.9423 - val_accuracy: 0.9207
Epoch 888/1000
382/382 [==============================] - 0s 88us/step - loss: 0.6165 - accuracy: 0.7749 - val_loss: 0.9173 - val_accuracy: 0.9268
Epoch 889/1000
382/382 [==============================] - 0s 92us/step - loss: 0.5353 - accuracy: 0.8220 - val_loss: 1.0893 - val_accuracy: 0.9146
Epoch 890/1000
382/382 [==============================] - 0s 88us/step - loss: 0.6279 - accuracy: 0.7880 - val_loss: 1.2982 - val_a

382/382 [==============================] - 0s 123us/step - loss: 0.6587 - accuracy: 0.7827 - val_loss: 0.9991 - val_accuracy: 0.9268
Epoch 940/1000
382/382 [==============================] - 0s 211us/step - loss: 0.5819 - accuracy: 0.8037 - val_loss: 1.2137 - val_accuracy: 0.9085
Epoch 941/1000
382/382 [==============================] - 0s 343us/step - loss: 0.5973 - accuracy: 0.8063 - val_loss: 1.0591 - val_accuracy: 0.9085
Epoch 942/1000
382/382 [==============================] - 0s 165us/step - loss: 0.5810 - accuracy: 0.8089 - val_loss: 0.9722 - val_accuracy: 0.9268
Epoch 943/1000
382/382 [==============================] - 0s 100us/step - loss: 0.5658 - accuracy: 0.7932 - val_loss: 1.0635 - val_accuracy: 0.9085
Epoch 944/1000
382/382 [==============================] - 0s 92us/step - loss: 0.6295 - accuracy: 0.7880 - val_loss: 0.9685 - val_accuracy: 0.9207
Epoch 945/1000
382/382 [==============================] - 0s 92us/step - loss: 0.6114 - accuracy: 0.7932 - val_loss: 0.8567 - va

382/382 [==============================] - 0s 103us/step - loss: 0.5437 - accuracy: 0.7932 - val_loss: 1.0061 - val_accuracy: 0.9268
Epoch 995/1000
382/382 [==============================] - 0s 157us/step - loss: 0.5732 - accuracy: 0.7906 - val_loss: 0.9902 - val_accuracy: 0.9268
Epoch 996/1000
382/382 [==============================] - 0s 172us/step - loss: 0.5529 - accuracy: 0.7880 - val_loss: 0.9788 - val_accuracy: 0.9329
Epoch 997/1000
382/382 [==============================] - 0s 181us/step - loss: 0.5973 - accuracy: 0.8063 - val_loss: 0.9812 - val_accuracy: 0.9329
Epoch 998/1000
382/382 [==============================] - 0s 153us/step - loss: 0.5545 - accuracy: 0.7932 - val_loss: 0.9756 - val_accuracy: 0.9329
Epoch 999/1000
382/382 [==============================] - 0s 141us/step - loss: 0.5863 - accuracy: 0.7801 - val_loss: 0.9889 - val_accuracy: 0.9268
Epoch 1000/1000
382/382 [==============================] - 0s 144us/step - loss: 0.6221 - accuracy: 0.7565 - val_loss: 0.9781 -

In [25]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

164/164 [==============================] - 0s 102us/step
over-sampling test accuracy: 93.29%


In [26]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [27]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 216 samples, validate on 93 samples
Epoch 1/100
216/216 [==============================] - 0s 531us/step - loss: 2.4433 - accuracy: 0.3426 - val_loss: 1.1483 - val_accuracy: 0.4194
Epoch 2/100
216/216 [==============================] - 0s 88us/step - loss: 0.8090 - accuracy: 0.7176 - val_loss: 0.5554 - val_accuracy: 0.7634
Epoch 3/100
216/216 [==============================] - 0s 117us/step - loss: 0.6426 - accuracy: 0.7315 - val_loss: 0.4116 - val_accuracy: 0.8387
Epoch 4/100
216/216 [==============================] - 0s 112us/step - loss: 0.4262 - accuracy: 0.8519 - val_loss: 0.3764 - val_accuracy: 0.8495
Epoch 5/100
216/216 [==============================] - 0s 111us/step - loss: 0.3447 - accuracy: 0.8889 - val_loss: 0.3569 - val_accuracy: 0.8710
Epoch 6/100
216/216 [==============================] - 0s 88us/step - loss: 0.3013 - accuracy: 0.9074 - val_loss: 0.3195 - val_accuracy: 0.8817
Epoch 7/100
216/216 [==============================] - 0s 106us/step - loss: 0.2615 - a

216/216 [==============================] - 0s 103us/step - loss: 0.0470 - accuracy: 0.9907 - val_loss: 0.1579 - val_accuracy: 0.9462
Epoch 57/100
216/216 [==============================] - 0s 118us/step - loss: 0.0451 - accuracy: 0.9907 - val_loss: 0.1554 - val_accuracy: 0.9462
Epoch 58/100
216/216 [==============================] - 0s 130us/step - loss: 0.0445 - accuracy: 0.9907 - val_loss: 0.1550 - val_accuracy: 0.9462
Epoch 59/100
216/216 [==============================] - 0s 102us/step - loss: 0.0429 - accuracy: 0.9907 - val_loss: 0.1579 - val_accuracy: 0.9462
Epoch 60/100
216/216 [==============================] - 0s 103us/step - loss: 0.0426 - accuracy: 0.9907 - val_loss: 0.1542 - val_accuracy: 0.9462
Epoch 61/100
216/216 [==============================] - 0s 116us/step - loss: 0.0412 - accuracy: 0.9907 - val_loss: 0.1488 - val_accuracy: 0.9462
Epoch 62/100
216/216 [==============================] - 0s 92us/step - loss: 0.0410 - accuracy: 0.9907 - val_loss: 0.1585 - val_accuracy:

In [29]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

93/93 [==============================] - 0s 103us/step
combination test accuracy: 94.62%


In [30]:
#### add dropout and regularization
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [31]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [32]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 216 samples, validate on 93 samples
Epoch 1/100
216/216 [==============================] - 0s 1ms/step - loss: 5.1437 - accuracy: 0.3704 - val_loss: 1.6135 - val_accuracy: 0.7742
Epoch 2/100
216/216 [==============================] - 0s 121us/step - loss: 4.9534 - accuracy: 0.6157 - val_loss: 1.2551 - val_accuracy: 0.7957
Epoch 3/100
216/216 [==============================] - 0s 218us/step - loss: 3.9940 - accuracy: 0.6806 - val_loss: 1.0811 - val_accuracy: 0.7957
Epoch 4/100
216/216 [==============================] - 0s 235us/step - loss: 4.1807 - accuracy: 0.6528 - val_loss: 0.9588 - val_accuracy: 0.7957
Epoch 5/100
216/216 [==============================] - 0s 215us/step - loss: 3.2360 - accuracy: 0.6898 - val_loss: 0.8606 - val_accuracy: 0.8387
Epoch 6/100
216/216 [==============================] - 0s 167us/step - loss: 3.5622 - accuracy: 0.7130 - val_loss: 0.8013 - val_accuracy: 0.8602
Epoch 7/100
216/216 [==============================] - 0s 151us/step - loss: 3.2181 - a

Epoch 57/100
216/216 [==============================] - 0s 256us/step - loss: 1.6079 - accuracy: 0.7593 - val_loss: 1.3284 - val_accuracy: 0.9032
Epoch 58/100
216/216 [==============================] - 0s 342us/step - loss: 1.4228 - accuracy: 0.7731 - val_loss: 1.4084 - val_accuracy: 0.9032
Epoch 59/100
216/216 [==============================] - 0s 248us/step - loss: 1.4220 - accuracy: 0.7824 - val_loss: 1.3088 - val_accuracy: 0.9032
Epoch 60/100
216/216 [==============================] - 0s 186us/step - loss: 1.6185 - accuracy: 0.7824 - val_loss: 1.1907 - val_accuracy: 0.9032
Epoch 61/100
216/216 [==============================] - 0s 206us/step - loss: 1.3419 - accuracy: 0.8380 - val_loss: 1.1268 - val_accuracy: 0.9032
Epoch 62/100
216/216 [==============================] - 0s 101us/step - loss: 1.5341 - accuracy: 0.7731 - val_loss: 1.1630 - val_accuracy: 0.9032
Epoch 63/100
216/216 [==============================] - 0s 104us/step - loss: 1.4243 - accuracy: 0.7685 - val_loss: 1.2450 -

In [33]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

93/93 [==============================] - 0s 106us/step
combination test accuracy: 93.55%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 91.46%


In [37]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 92.47%


In [39]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.76923077 0.84615385 0.93421053 0.81333333 0.93333333]
0.8592523616734142


In [40]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.97777778 0.97727273 0.97674419 0.92857143 0.92857143]
0.9577875096479748
